# 类与对象

## 改变对象的字符串显示

In [1]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return 'Pair({0.x!r},{0.y!r})'.format(self)

    def __str__(self):
        return '({0.x!r},{0.y!r})'.format(self)

p = Pair(3, 4)
print(p)
p

(3,4)


Pair(3,4)

## 自定义字符串的格式化

In [2]:
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
}

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

d = Date(2020, 7, 18)
print(format(d))
print(format(d, 'mdy'))

2020-7-18
7/18/2020


## 让对象支持上下文管理协议

In [3]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type = SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.sock = None

    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Connection already existed')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock

    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

from functools import partial

conn = LazyConnection(('www.python.org', 80))

with conn as s:
    # conn.__enter_()执行
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))
    print(resp)
    # conn.__exit__()执行

b'HTTP/1.1 301 Moved Permanently\r\nServer: Varnish\r\nRetry-After: 0\r\nLocation: https://www.python.org/index.html\r\nContent-Length: 0\r\nAccept-Ranges: bytes\r\nDate: Sat, 18 Jul 2020 07:20:45 GMT\r\nVia: 1.1 varnish\r\nConnection: close\r\nX-Served-By: cache-hnd18722-HND\r\nX-Cache: HIT\r\nX-Cache-Hits: 0\r\nX-Timer: S1595056845.319131,VS0,VE0\r\nStrict-Transport-Security: max-age=63072000; includeSubDomains\r\n\r\n'


## 创建大量对象时节省内存方法

In [4]:
class Date:
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

当定义 __slots__ 后，Python 就会为实例使用一种更加紧凑的内部表示。实例通过一个很小的固定大小的数组来构建，而不是为每个实例定义一个字典，这跟元组或列表很类似。

## 在类中封装属性名

In [5]:
class A:
    def __init__(self):
        self._internal = 0 # 一个内部属性
        self.public = 1 # 一个公开属性

    def public_method(self):
        pass

    def _internal_method(self):
        pass

In [6]:
class B:
    def __init__(self):
        self.__private = 0
        
    def __private_method(self):
        pass
    
    def public_method(self):
        pass
        self.__private_method()

使用双下划线开始会导致访问名称变成其他形式。比如，在前面的类 B 中，私有属性会被分别重命名为 _B__private 和_B__private_method 。这时候你可能会问这样重命名的目的是什么，答案就是继承——这种属性通过继承是无法被覆盖的。

In [7]:
class C(B):
    def __init__(self):
        super().__init__()
        self.__private = 1 # Does not override B.__private
    
    # Does not override B.__private_method()
    def __private_method(self):
        pass

## 创建可管理的属性

In [8]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name

    @property
    def first_name(self):
        return self._first_name

    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise RuntimeError('Expected a string')
        self._first_name = value

    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")

In [9]:
a = Person('chun')
a.first_name

'chun'

In [10]:
a.first_name = 42

RuntimeError: Expected a string

In [11]:
del a.first_name

AttributeError: Can't delete attribute

Properties 还是一种定义动态计算 attribute 的方法。这种类型的 attributes 并不会被实际的存储，而是在需要的时候计算出来。

## 调用父类方法

In [12]:
# 为了调用父类 (超类) 的一个方法，可以使用 super() 函数
class A:
    def spam(self):
        print('A.spam')

class B(A):
    def spam(self):
        print('B.spam')
        super().spam()

b = B()
b.spam()

B.spam
A.spam


In [13]:
# super() 函数的一个常见用法是在 __init__() 方法中确保父类被正确的初始化了
class A:
    def __init__(self):
        self.x = 0

class B(A):
    def __init__(self):
        super().__init__()
        self.y = 1

b = B()
print(b.x , b.y)

0 1


In [14]:
# super() 的另外一个常见用法出现在覆盖 Python 特殊方法的代码中
class Proxy:
    def __init__(self, obj):
        self._obj = obj
    
    def __getattr__(self, name):
        return getattr(self._obj, name)

    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value)
        else:
            setattr(self._obj, name, value)

## 子类中扩展property

In [15]:
class Person:
    def __init__(self, name):
        self.name = name

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise RuntimeError('Expected a string')
        self._name = value

    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")

In [16]:
class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        return super().name

    @name.setter
    def name(self, value):
        print('Setting name to ', value)
        super(SubPerson, SubPerson).name.__set__(self, value)

    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson, SubPerson).name.__delete__(self)

In [17]:
s = SubPerson('Guido')

Setting name to  Guido


In [18]:
s.name

Getting name


'Guido'

In [19]:
s.name = 'Larry'

Setting name to  Larry


## 创建新的类或实例属性

In [20]:
class Integer:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Expected an int')
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        del instance.__dict__[self.name] 

In [21]:
class Point:
    x = Integer('x')
    y = Integer('y')
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

p = Point(2, 3)
print(p.x, p.y)

2 3


## 使用延迟计算属性

In [22]:
class lazyproperty:
    def __init__(self, func):
        self.func = func

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value

import math
class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2
        
    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius

In [23]:
c = Circle(4.0)
print(c.radius)
print(c.area)
print(c.perimeter)
print(c.perimeter)

4.0
Computing area
50.26548245743669
Computing perimeter
25.132741228718345
25.132741228718345


## 简化数据结构的初始化

In [24]:
import math

# 在一个基类中写一个公用的__init__()函数

class Structure1:
    _fields = []

    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
        for name, value in zip(self._fields, args):
            setattr(self, name, value)

# 然后继承即可
class Stock(Structure1):
    _fields = ['name', 'shares', 'price']

class Point(Structure1):
    _fields = ['x', 'y']

class Circle(Structure1):
    _fields = ['radius']
    
    def area(self):
        return math.pi * self.radius ** 2

In [25]:
s = Stock('ACME', 50, 91.1)
p = Point(2, 3)
c = Circle(4.5)

如果还想支持关键字参数，可以将关键字参数设置为实例属性：

In [26]:
class Structure2:
    _fields = []

    def __init__(self, *args, **kwargs):
        if len(args) > len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
        for name, value in zip(self._fields, args):
            setattr(self, name, value)
        for name in self._fields[len(args):]:
            setattr(self, name, kwargs.pop(name))
        if kwargs:
            raise TypeError('Invalid argument(s): {}'.format(','.join(kwargs)))

class Stock(Structure2):
    _fields = ['name', 'shares', 'price']

s1 = Stock('ACME', 50, 91.1)
s2 = Stock('ACME', 50, price=91.1)
s3 = Stock('ACME', shares=50, price=91.1)

In [27]:
# 还能将不在 _fields 中的名称加入到属性中去：


class Structure3:
    _fields = []

    def __init__(self, *args, **kwargs):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
        for name, value in zip(self._fields, args):
            setattr(self, name, value)
        extra_args = kwargs.keys() - self._fields
        for name in extra_args:
            setattr(self, name, kwargs.pop(name))
        if kwargs:
            raise TypeError('Duplicate values for {}'.format(','.join(kwargs)))

class Stock(Structure3):
    _fields = ['name', 'shares', 'price']

s1 = Stock('ACME', 50, 91.1)
s2 = Stock('ACME', 50, 91.1, date='8/2/2012')

## 定义接口或者抽象基类

In [28]:
from abc import ABCMeta, abstractmethod

class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self, maxbytes=-1):
        pass
    @abstractmethod
    def wirte(self, data):
        pass

try:
    # 抽象类不能直接被实例化
    a = IStream()
except TypeError:
    print('TypeError')

TypeError


In [29]:
# 抽象类的目的就是让别的类继承它并实现特定的抽象方法
class SocketStream(IStream):
    def read(self, maxbytes=-1):
        pass

    def wirte(self, data):
        pass

In [30]:
# 抽象基类的一个主要用途是在代码中检查某些类是否为特定类型，实现了特定接口

def serialize(obj, stream):
    if not isinstance(stream, IStream):
        raise TypeError('Expected an IStream')
    pass

In [31]:
# 可以通过注册的方式来让某个类实现抽象基类

import io

IStream.register(io.IOBase)

f = open('data/somefile.txt')
print(isinstance(f, IStream))
f.close()

True


@abstractmethod 还能注解静态方法、类方法和 properties 。你只需保证这个注解紧靠在函数定义前即可

## 实现数据模型的类型约束

In [32]:
class Descriptor:
    def __init__(self, name=None, **opts):
        self.name = name
        for key, val in opts.items():
            setattr(self, key, val)
    
    def __set__(self, instance, val):
        instance.__dict__[self.name] = val

class Typed(Descriptor):
    expected_type = type(None)

    def __set__(self, instance, val):
        if not isinstance(val, self.expected_type):
            raise  TypeError('expected ' + str(self.expected_type))
        super().__set__(instance, val)

class Unsigned(Descriptor):
    def __set__(self, instance, val):
        if val < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, val)

class MaxSized(Descriptor):
    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super().__init__(name, **opts)

    def __set__(self, instance, val):
        if len(val) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super().__set__(instance, val)

class Integer(Typed):
    expected_type = int
class UnsignedInteger(Integer, Unsigned):
    pass
class Float(Typed):
    expected_type = float
class UnsignedFloat(Float, Unsigned):
    pass
class String(Typed):
    expected_type = str
class SizedString(String, MaxSized):
    pass

In [33]:
class Stock:
    name = SizedString('name', size=8)
    shares = UnsignedInteger('shares')
    price = UnsignedFloat('price')
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [34]:
s = Stock('ACME', 70, 90.0)

In [35]:
s.name = 'AAAAAAAAAAAAAAAAAAAAAAAA'

ValueError: size must be < 8

In [36]:
s.shares = -10

ValueError: Expected >= 0

In [37]:
# 也可以用装饰器实现上述功能

def check_attributes(**kwargs):
    def decorate(cls):
        for key, val in kwargs.items():
            if isinstance(val, Descriptor):
                val.name = key
                setattr(cls, key, val)
            else:
                setattr(cls, key, val(key))
        return cls
    return decorate

@check_attributes(name=SizedString(size=8),
                  shares=UnsignedInteger,
                  price=UnsignedFloat)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

In [38]:
# 或者使用元类

class checkedmeta(type):
    def __new__(cls, clsname, bases, methods):
        for key, value in methods.items():
            if isinstance(value, Descriptor):
                value.name = key
        return type.__new__(cls, clsname, bases, methods)
    
class Stock(metaclass=checkedmeta):
    name = SizedString(size=8)
    shares = UnsignedInteger()
    price = UnsignedFloat()
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

## 实现自定义容器

In [39]:
import collections
import bisect

class SortedItems(collections.Sequence):
    def __init__(self, initial=None):
        self._items = sorted(initial) if initial is not None else []
    
    def __getitem__(self, idx):
        return self._items[idx]

    def __len__(self):
        return len(self._items)

    def add(self, item):
        bisect.insort(self._items, item)

items = SortedItems([5, 1, 3])
print(list(items))
print(items[0], items[-1])
items.add(2)
print(list(items))

[1, 3, 5]
1 5
[1, 2, 3, 5]


## 属性的代理访问

In [40]:
# 代理是一种编程模式，它将某个操作转移给另外一个对象来实现。

class A:
    def spam(self, x):
        pass
    
    def foo(self):
        pass

class B1:
    """简单的代理"""
    def __init__(self):
        self._a = A()

    def spam(self, x):
        return self._a.spam(x)

    def foo(self):
        return self._a.foo()

    def bar(self):
        pass

class B2:
    """ 使用__getattr__ 的代理，代理方法比较多时候"""
    def __init__(self):
        self._a = A()
    
    def bar(self):
        pass

    def __getattr__(self, name):
        """ 这个方法在访问的 attribute 不存在的时候被调用
        the __getattr__() method is actually a fallback method
        that only gets called when an attribute is not found"""
        return getattr(self._a, name)

## 在类中定义多个构造器

In [41]:
import time

class Date:
    """方法一：使用类方法"""
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    
    @classmethod
    def today(cls):
        t = time.localtime()
        return cls(t.tm_year, t.tm_mon, t.tm_mday)

    def __str__(self):
        return '{}-{}-{}'.format(self.year, self.month, self.day)

print(Date(2020, 7, 18))
print(Date.today())

2020-7-18
2020-7-18


## 创建不调用init方法的实例

In [42]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    
    def __str__(self):
        return '{}-{}-{}'.format(self.year, self.month, self.day)

d = Date.__new__(Date)
data = {'year':2020, 'month':7, 'day':18}
for key, val in data.items():
    setattr(d, key, val)
print(d)

2020-7-18


## 利用Mixins扩展类功能

In [43]:
class LoggedMappingMixin:
    """
    Add logging to get/set/delete operations for debugging.
    """
    __slots__ = ()
    # 混入类都没有实例变量，因为直接实例化混入类没有任何意义
    def __getitem__(self, key):
        print('Getting ' + str(key))
        return super().__getitem__(key)

    def __setitem__(self, key, value):
        print('Setting {} = {!r}'.format(key, value))
        return super().__setitem__(key, value)

    def __delitem__(self, key):
        print('Deleting ' + str(key))
        return super().__delitem__(key)

class SetOnceMappingMixin:
    '''
    Only allow a key to be set once.
    '''
    __slots__ = ()
    def __setitem__(self, key, value):
        if key in self:
            raise KeyError(str(key) + ' already set')
        return super().__setitem__(key, value)

class StringKeysMappingMixin:
    '''
    Restrict keys to strings only
    '''
    __slots__ = ()
    def __setitem__(self, key, value):
        if not isinstance(key, str):
            raise TypeError('keys must be strings')
        return super().__setitem__(key, value)

In [44]:
from collections import defaultdict
class SetOnceDefaultDict(SetOnceMappingMixin, defaultdict):
    pass
d = SetOnceDefaultDict(list)
d['x'].append(2)
d['x'].append(3)
d['x'] = 23

KeyError: 'x already set'

## 实现状态对象或者状态机

In [45]:
class Connection:
    def __init__(self):
        self.state = 'CLOSED'

    def read(self):
        if self.state != 'OPEN':
            raise RuntimeError('Not open')
        print('reading')
    
    def write(self, data):
        if self.state != 'OPEN':
            raise RuntimeError('Not open')
        print('writing')
    
    def open(self):
        if self.state == 'OPEN':
            raise RuntimeError('Already open')
        self.state = 'OPEN'
    
    def close(self):
        if self.state == 'CLOSED':
            raise RuntimeError('Already closed')
        self.state = 'CLOSED'

In [47]:
class Connection1:
    """ 新方案——对每个状态定义一个类"""
    def __init__(self):
        self.new_state(ClosedConnectionState)

    def new_state(self, newstate):
        self._state = newstate
    
    def read(self):
        return self._state.read(self)
    
    def write(self, data):
        return self._state.write(self, data)

    def open(self):
        return self._state.open(self)

    def close(self):
        return self._state.close(self)

class ConnectionState:
    @staticmethod
    def read(conn):
        raise NotImplementedError()
    @staticmethod
    def write(conn, data):
        raise NotImplementedError()
    @staticmethod
    def open(conn):
        raise NotImplementedError()
    @staticmethod
    def close(conn):
        raise NotImplementedError()

# Implementation of different states
class ClosedConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        raise RuntimeError('Not open')
    @staticmethod
    def write(conn, data):
        raise RuntimeError('Not open')
    @staticmethod
    def open(conn):
        conn.new_state(OpenConnectionState)
    @staticmethod
    def close(conn):
        raise RuntimeError('Already closed')

class OpenConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        print('reading')
    @staticmethod
    def write(conn, data):
        print('writing')
    @staticmethod
    def open(conn):
        raise RuntimeError('Already open')
    @staticmethod
    def close(conn):
        conn.new_state(ClosedConnectionState)

In [51]:
c = Connection1()
c.open()
c.read()
c.write("hello")
c.close()

reading
writing


## 通过字符串调用对象方法

In [53]:
import math

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return 'Point({!r:},{!r:})'.format(self.x, self.y)
    
    def distance(self, x, y):
        return math.hypot(self.x - x, self.y - y)

p = Point(2, 3)
print(getattr(p, 'distance')(0,0))

import operator
print(operator.methodcaller('distance', 0, 0)(p))

3.605551275463989
3.605551275463989


In [56]:
points = [
    Point(1, 2),
    Point(3, 0),
    Point(10, -3),
    Point(-5, -7),
    Point(-1, 8),
    Point(3, 2)
    ]

points.sort(key=operator.methodcaller('distance', 0, 0))
print(points)

[Point(1,2), Point(3,0), Point(3,2), Point(-1,8), Point(-5,-7), Point(10,-3)]


## 实现访问者模式

In [58]:
class Node:
    pass

class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value

In [59]:
# Representation of 1 + 2 * (3 - 4) / 5
t1 = Sub(Number(3), Number(4))
t2 = Mul(Number(2), t1)
t3 = Div(t2, Number(5))
t4 = Add(Number(1), t3)

In [60]:
class NodeVisitor:
    def visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)
    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_' + type(node).__name_))

In [61]:
class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value
    
    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)
    
    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)
    
    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)
    
    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)
    
    def visit_Negate(self, node):
        return -node.operand

In [62]:
e = Evaluator()
e.visit(t4)

0.6

In [63]:
class StackCode(NodeVisitor):
    def generate_code(self, node):
        self.instructions = []
        self.visit(node)
        return self.instructions
    
    def visit_Number(self, node):
        self.instructions.append(('PUSH', node.value))
    
    def binop(self, node, instruction):
        self.visit(node.left)
        self.visit(node.right)
        self.instructions.append((instruction,))
    
    def visit_Add(self, node):
        self.binop(node, 'ADD')
    
    def visit_Sub(self, node):
        self.binop(node, 'SUB')
    
    def visit_Mul(self, node):
        self.binop(node, 'MUL')
    
    def visit_Div(self, node):
        self.binop(node, 'DIV')
    
    def unaryop(self, node, instruction):
        self.visit(node.operand)
        self.instructions.append((instruction,))
    
    def visit_Negate(self, node):
        self.unaryop(node, 'NEG')

In [64]:
s = StackCode()
s.generate_code(t4)

[('PUSH', 1),
 ('PUSH', 2),
 ('PUSH', 3),
 ('PUSH', 4),
 ('SUB',),
 ('MUL',),
 ('PUSH', 5),
 ('DIV',),
 ('ADD',)]

## 不用递归实现访问者模式

In [65]:
import types

class Node:
    pass

class NodeVisitor:
    def visit(self, node):
        stack = [node]
        last_result = None
        while stack:
            try:
                last = stack[-1]
                if isinstance(last, types.GeneratorType):
                    stack.append(last.send(last_result))
                    last_result = None
                elif isinstance(last, Node):
                    stack.append(self._visit(stack.pop()))
                else:
                    last_result = stack.pop()
            except StopIteration:
                stack.pop()
        return last_result
    
    def _visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)
    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_' + type(node).__name_))

In [69]:
class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value

class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value

    def visit_Add(self, node):
        yield (yield node.left) + (yield node.right)
    
    def visit_Sub(self, node):
        yield (yield node.left) - (yield node.right)

    def visit_Mul(self, node):
        yield (yield node.left) * (yield node.right)

    def visit_Div(self, node):
        yield (yield node.left) / (yield node.right)

    def visit_Negate(self, node):
        yield - (yield node.operand)

In [70]:
t1 = Sub(Number(3), Number(4))
t2 = Mul(Number(2), t1)
t3 = Div(t2, Number(5))
t4 = Add(Number(1), t3)
e = Evaluator()
print(e.visit(t4))

0.6


## 循环引用数据结构的内存管理

In [71]:
import weakref
class Node:
    def __init__(self, value):
        self.value = value
        self._parent = None
        self.children = []

    def __repr__(self):
        return 'Node({!r:})'.format(self.value)

    @property
    def parent(self):
        return None if self._parent is None else self._parent()
    
    @parent.setter
    def parent(self, node):
        self._parent = weakref.ref(node)
        
    def add_child(self, child):
        self.children.append(child)
        child.parent = self

In [72]:
root = Node('parent')
c1 = Node('child')
root.add_child(c1)
print(c1.parent)
del root
print(c1.parent)

Node('parent')
None


## 让类支持比较操作

In [73]:
from functools import total_ordering
class Room:
    def __init__(self, name, length, width):
        self.name = name
        self.length = length
        self.width = width
        self.square_feet = self.length * self.width
    
@total_ordering
class House:
    def __init__(self, name, style):
        self.name = name
        self.style = style
        self.rooms = list()
    
    @property
    def living_space_footage(self):
        return sum(r.square_feet for r in self.rooms)
    
    def add_room(self, room):
        self.rooms.append(room)
    
    def __str__(self):
        return '{}: {} square foot {}'.format(self.name,self.living_space_footage,self.style)

    def __eq__(self, other):
        return self.living_space_footage == other.living_space_footage
    def __lt__(self, other):
        return self.living_space_footage < other.living_space_footage

In [74]:
h1 = House('h1', 'Cape')
h1.add_room(Room('Master Bedroom', 14, 21))
h1.add_room(Room('Living Room', 18, 20))
h1.add_room(Room('Kitchen', 12, 16))
h1.add_room(Room('Office', 12, 12))
h2 = House('h2', 'Ranch')
h2.add_room(Room('Master Bedroom', 14, 21))
h2.add_room(Room('Living Room', 18, 20))
h2.add_room(Room('Kitchen', 12, 16))
h3 = House('h3', 'Split')
h3.add_room(Room('Master Bedroom', 14, 21))
h3.add_room(Room('Living Room', 18, 20))
h3.add_room(Room('Office', 12, 16))
h3.add_room(Room('Kitchen', 15, 17))
houses = [h1, h2, h3]
print('Is h1 bigger than h2?', h1 > h2) 
print('Is h2 smaller than h3?', h2 < h3) 
print('Is h2 greater than or equal to h1?', h2 >= h1) 
print('Which one is biggest?', max(houses))
print('Which is smallest?', min(houses))

Is h1 bigger than h2? True
Is h2 smaller than h3? True
Is h2 greater than or equal to h1? False
Which one is biggest? h3: 1101 square foot Split
Which is smallest? h2: 846 square foot Ranch


## 创建缓存实例

In [75]:
class Spam:
    def __init__(self, name):
        self.name = name

# Caching support
import weakref
_spam_cache = weakref.WeakValueDictionary()
def get_spam(name):
    if name not in _spam_cache:
        s = Spam(name)
        _spam_cache[name] = s
    else:
        s = _spam_cache[name]
    return s

In [76]:
a = get_spam('foo')
b = get_spam('bar')
print(a is b)
c = get_spam('foo')
print(a is c)

False
True
